首先导入相关的库，注意安装autogluon的时候记得关VPN，不然pycharm老是在报错

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

读取数据，这里使用的是m4的小时数据集，总共有148060行数据，每行数据有三列
一列是物品名称，item_id,一列是时间戳，timestamp，一列是目标值，target

In [17]:
df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")
df.head()

,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0


# autogluon对数据格式的要求
1. 独特的时间序列id，可为int或者str类型
2. 时间戳必须是满足pd.Timestamp的格式 2018-03-17 22:15:15
3. 数值型的目标值target
至少要有这三列，其他的列可以有，也可以没有
但是我们在创造TimeSeriesDataFrame的时候，需要指定哪一列是时间序列id，哪一列是时间戳，哪一列是目标值
如果格式不匹配，会报错

In [18]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp"
)
train_data.head()

target
item_id timestamp                  
H1      1750-01-01 00:00:00   605.0
        1750-01-01 01:00:00   586.0
        1750-01-01 02:00:00   586.0
        1750-01-01 03:00:00   559.0
        1750-01-01 04:00:00   511.0

我们可以看到，train_data的格式是TimeSeriesDataFrame，这是autogluon自己定义的数据格式
H1表示的是一只股票的时间序列，H2表示的是另一只股票的时间序列
Autogluon单独为每个时间序列生成预测,并不是对不同的序列之间的交互进行预测
同时为了预测未来的值我们需要创建一个time series predictor
prediction_length表示的是我们要预测的未来的时间长度,这里我们预测未来48个小时的值
path表示的是我们训练好的模型的保存路径
target表示的是我们要预测的目标值
eval_metric表示的是我们的评估指标，这里我们使用的是MASE
presets表示训练的质量,还有选择是high_quality但是训练时间会更长
time_limit表示的是训练的时间限制，这里我们设置的是600秒
autogluon会自动选择最好的模型，我们也可以自己指定模型

In [19]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MASE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'MASE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'num_val_windows': 1,
 'prediction_length': 48,
 'random_seed': None,
 'target': 'target',
 'time_limit': 600,
 'verbosity': 2}
Provided training data set with 148060 rows, 200 items (item = single time series). Average time series length is 740.3. Data frequency is 'H'.
AutoGluon will save models to autogluon-m4-hourly\
AutoGluon will gauge predictive performance using evaluation metric: 'MASE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'target'

Starting training. Start time is 2023-08-22 20:54:12
Models that will be 

autogluon还会产生概率预测,它还提供分布的置信区间,让我们了解结果的可能范围
for example,0.1表示你有10%的可能性会低于这个值(有点类似正态分布

In [ ]:
predictions = predictor.predict(train_data)
predictions.head()

我们现在可视化一下预测的结果和真实值的对比
我们绘制 mean forecast 以及0.1和0.9所在的置信区间

In [ ]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

plt.figure(figsize=(20, 3))

item_id = "H1"
y_past = train_data.loc[item_id]["target"]
y_pred = predictions.loc[item_id]
y_test = test_data.loc[item_id]["target"][-48:]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval"
)
plt.legend();

# 评估不同模型的效果
我们可以使用autogluon的leaderboard来评估不同模型的效果

In [ ]:
# The test score is computed using the last
# prediction_length=48 timesteps of each time series in test_data
predictor.leaderboard(test_data, silent=True)

# 总结
1. autogluon不需要调参数
2. 数据结构一定要满足time series dataframe的要求